In [64]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
df_FIFA = pd.read_csv('/content/drive/MyDrive/FIFA dataset/male_players (legacy).csv', na_values = '-', low_memory=False)
df_player22 = pd.read_csv('/content/drive/MyDrive/FIFA dataset/players_22-1.csv', na_values = '-', low_memory=False)
df_FIFA.shape



(161583, 110)

In [67]:
df_FIFA

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,/player/269011/ziye-zhao/230002,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,269019,/player/269019/junjie-wu/230002,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,271093,/player/271093/kailin-barlow/230002,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,271555,/player/271555/gideon-tetteh/230002,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


In [68]:
def DropColumns(df_FIFA):
  LessMissing =[]
  MoreMissing = []
  for i in df_FIFA.columns:
    if "url" in i:
        MoreMissing.append(i)
        # use 0.3 for the 30%
    if((df_FIFA[i].isnull().sum())<(0.3*(df_FIFA.shape[0]))):
      LessMissing.append(i)
    else:
      MoreMissing.append(i)
  # df_FIFA = df_FIFA[LessMissing]
  df_FIFA.drop(MoreMissing, axis = 1, inplace = True)
  return df_FIFA


In [82]:
def RemoveNANs(df_FIFA2):
  df_FIFA2 = DropColumns(df_FIFA2)
  # Separating features into numeric and categorical variable
  numeric_data = df_FIFA2.select_dtypes(include = np.number)
  non_numeric_data = df_FIFA2.select_dtypes(include = ['object'])
  label_encoders = {}
  # handling missing values in the non numeric data
  for column in non_numeric_data.columns:
    le = LabelEncoder()
    non_numeric_data[column] = le.fit_transform(non_numeric_data[column].astype(str))
    label_encoders[column] = le
    # handling missing values in the non numeric data
  imp = IterativeImputer(max_iter = 10, random_state = 0)
  numeric_data = pd.DataFrame(np.round(imp.fit_transform(numeric_data)), columns = numeric_data.columns)
  Y = numeric_data['overall'].astype(int)
  X = pd.concat([numeric_data, non_numeric_data], axis=1)
  # Check for classes with only one sample
  value_counts = Y.value_counts()
  single_sample_classes = value_counts[value_counts == 1].index.tolist()

  # Remove rows with target values belonging to single-sample classes
  X = X[~Y.isin(single_sample_classes)]
  Y = Y[~Y.isin(single_sample_classes)]
  correlations = X.corr()['overall'].sort_values(ascending=False)
  strong_correlations = correlations[:7]
  strong_correlations_columns  = strong_correlations.index.tolist()
  strong_correlations_columns
  strong_correlations_columns.remove('overall')
  X = X[strong_correlations_columns]
  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  return X, Y




In [83]:
print(X.shape)

(19239, 6)


In [87]:
# Calculate the correlation between each variable and player's overall rating
correlations[0:7]

overall                1.000000
movement_reactions     0.845753
mentality_composure    0.699994
potential              0.695362
wage_eur               0.611981
rcm                    0.607929
cm                     0.607929
Name: overall, dtype: float64

In [89]:
X, Y = RemoveNANs(df_player22)
Y

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  for feat_idx in ordered_idx:


2        91
3        91
4        91
5        91
6        91
         ..
19234    47
19235    47
19236    47
19237    47
19238    47
Name: overall, Length: 19237, dtype: int64

In [90]:
X.shape

(19237, 6)

In [91]:
Xtrain, Xtest, Ytrain,Ytest = train_test_split(X, Y,test_size = 0.2, random_state=42, stratify=Y)
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((15389, 6), (3848, 6), (15389,), (3848,))

# Base Training Phase

## 1] Ridge Model

In [92]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Train the model
model = Ridge() # you replace put ElasticNet or lasso on the Ridge function to compare and  train
model.fit(Xtrain, Ytrain)

#Test the model
y_pred = model.predict(Xtest)

print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 2.236449903283544,
           Mean Squared Error = 8.607090890627614,
           Root Mean Squared Error = 2.9337843974340743,
           r2 score = 0.7833580961423253
           


## 2] ElasticNet Model

In [93]:
from sklearn.linear_model import ElasticNet
#Train the model
model = ElasticNet() # you replace put ElasticNet or lasso on the Ridge function to compare and  train
model.fit(Xtrain, Ytrain)

#Test the model
y_pred = model.predict(Xtest)

print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 2.6381281489261186,
           Mean Squared Error = 11.69503493662282,
           Root Mean Squared Error = 3.419800423507609,
           r2 score = 0.4711771532033835
           


## 3] Lasso Model

In [94]:
from sklearn.linear_model import Lasso
#Train the model
model = Lasso() # you replace put ElasticNet or lasso on the Ridge function to compare and  train
model.fit(Xtrain, Ytrain)

#Test the model
y_pred = model.predict(Xtest)

print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 2.4476786331520195,
           Mean Squared Error = 9.9197771770413,
           Root Mean Squared Error = 3.1495677762260175,
           r2 score = 0.6272020585500456
           


## 4] Linear Regression Model

In [95]:
from sklearn.linear_model import LinearRegression

# Train and evaluate linear regression model
linear_model = LinearRegression()
linear_model.fit(Xtrain, Ytrain)
y_pred_linear = linear_model.predict(Xtest)
print(f""" Mean Absolute Error = {mean_absolute_error(y_pred_linear, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred_linear, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred_linear, Ytest))},
           r2 score = {r2_score(y_pred_linear, Ytest)}
           """)

 Mean Absolute Error = 2.236457972276683,
           Mean Squared Error = 8.607141622762162,
           Root Mean Squared Error = 2.9337930436147266,
           r2 score = 0.7833732794919475
           


## 5] Decision Tree Regressor Model

In [96]:
from sklearn.tree import DecisionTreeRegressor

# Train and evaluate  Decision Tree Regressor model
dtree = DecisionTreeRegressor(max_depth = 12) #Change it to 7 and 12 and compare the values
dtree.fit(Xtrain, Ytrain)
y_pred = dtree.predict(Xtest)
print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 1.886666285715926,
           Mean Squared Error = 7.204782557834901,
           Root Mean Squared Error = 2.6841726020945265,
           r2 score = 0.8327855024678497
           


# Training and Evaluating Ensemble Models

In [97]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from xgboost import XGBRegressor

In [ ]:
rf_model = RandomForestRegressor(random_state=42)
# Define parameter grids for RandomForestRegressor model
rf_param_grid = {
    'n_estimators': [500, 600],
    'max_depth': [2, 10, 20],
    'min_samples_split': [2, 5]
}
# Perform grid search with cross-validation
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(Xtrain, Ytrain)
rf_r2 = r2_score(Ytest, rf_grid_search.predict(Xtest))
rf_best_model = rf_grid_search.best_estimator_
rf_best_model.fit(Xtrain, Ytrain)
rf_y_pred = rf_best_model.predict(Xtest)
rf_mse = mean_squared_error(Ytest, rf_y_pred)
cross_val_score(rf_best_model, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')

print("Random Forest MSE:", rf_mse)
print("Random Forest R2:", rf_r2)
print("Random Forest best parameters:", rf_grid_search.best_params_)
print("Random Forest best score:", rf_grid_search.best_score_)
print("Random Forest best estimator:", rf_grid_search.best_estimator_)
print("Random Forest feature importances:", rf_grid_search.best_estimator_.feature_importances_)


In [38]:
xgb_model = XGBRegressor(random_state=42)
xgb_param_grid = {
    'n_estimators': [600, 650],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_grid = GridSearchCV(xgb_model, xgb_param_grid, cv=5, scoring='neg_mean_squared_error')
xgb_grid.fit(Xtrain, Ytrain)
xgb_grid_search = xgb_grid.best_estimator_
xgb_grid_search.fit(Xtrain, Ytrain)
xgb_y_pred = xgb_grid_search.predict(Xtest)
xgb_mse = mean_squared_error(Ytest, xgb_y_pred)
xgb_cross_val = cross_val_score(xgb_grid_search, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')
xgb_r2 = r2_score(Ytest, xgb_y_pred)

print("XGBoost MSE:", xgb_mse)
print("XGBoost R2:", xgb_r2)
print("XGBoost best score:", xgb_grid.best_score_)
print("XGBoost best estimator:", xgb_grid.best_estimator_)
print("XGBoost feature importances:", xgb_grid.best_estimator_.feature_importances_)
print("XGBoost cross validation score:", xgb_cross_val)


XGBoost MSE: 4.40201654518403
XGBoost R2: 0.9111842877249092
XGBoost best score: -4.464424588597028
XGBoost best estimator: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
XGBoost feature importances: [0.18875691 0.03650922 0.06344959 0.68400955 0.02727478 0.
 0.        ]
XGBoost cross validation score: [-

In [41]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}
gb_grid = GridSearchCV(gb_model, gb_param_grid, cv=5, scoring='neg_mean_squared_error')
# gb_grid_search = GridSearchCV(estimator=gb_model, param_grid=gb_param_grid, cv=5, scoring='neg_mean_squared_error')
gb_grid.fit(Xtrain, Ytrain)
gb_grid_search = gb_grid.best_estimator_
gb_grid_search.fit(Xtrain, Ytrain)
gb_y_pred = gb_grid_search.predict(Xtest)
gb_mse = mean_squared_error(Ytest, gb_y_pred)
gb_cross_val = cross_val_score(gb_grid_search, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')
gb_r2 = r2_score(Ytest, gb_y_pred)


print("Gradient Boosting MSE:", gb_mse)
print("Gradient Boosting R2:", gb_r2)
print("Gradient Boosting cross validation score:", gb_cross_val)
#print("Gradient Boosting feature importances:", gb_grid_search.feature_importances_)
print("Gradient Boosting feature importances:", gb_grid_search.feature_importances_) # Changed line
print("Gradient Boosting best parameters:", gb_grid.best_params_)
print("Gradient Boosting best score:", gb_grid.best_score_)
print("Gradient Boosting best estimator:", gb_grid.best_estimator_)



Gradient Boosting MSE: 4.379827914838064
Gradient Boosting R2: 0.9116319686884762
Gradient Boosting cross validation score: [-4.47063597 -4.3569076  -4.50318918 -4.44685793 -4.45732284]
Gradient Boosting feature importances: [0.28084001 0.05249239 0.08342285 0.54632814 0.0104786  0.01334175
 0.01309627]
Gradient Boosting best parameters: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200}
Gradient Boosting best score: -4.446982703892174
Gradient Boosting best estimator: GradientBoostingRegressor(max_depth=10, n_estimators=200, random_state=42)


In [40]:
adb_model = AdaBoostRegressor(random_state=42)
adb_param_grid = {
    'n_estimators': [700, 800],
    'learning_rate': [0.01, 0.1, 0.2]
}
adb_grid_search = GridSearchCV(estimator=adb_model, param_grid=adb_param_grid, cv=5, scoring='neg_mean_squared_error')
adb_grid_search.fit(Xtrain, Ytrain)
adb_best_model = adb_grid_search.best_estimator_
adb_best_model.fit(Xtrain, Ytrain)
adb_y_pred = adb_best_model.predict(Xtest)
adb_mse = mean_squared_error(Ytest, adb_y_pred)
adb_cross_val = cross_val_score(adb_best_model, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')
adb_r2 = r2_score(Ytest, adb_y_pred)
# adb_best_params = adb_grid_search.best_params_
# adb_best_score = adb_grid_search.best_score_
# adb_best_estimator = adb_grid_search.best_estimator_

print("AdaBoosting Best Model:", adb_best_model)
print("AdaBoosting MSE:", adb_mse)
print("AdaBoosting R2:", adb_r2)
print("AdaBoosting cross validation score:", adb_cross_val)
# print("AdaBoosting feature importances:", adb_grid_search.feature_importances_)
print("AdaBoosting feature importances:", adb_best_model.feature_importances_)
print("Gradient Boosting best parameters:", adb_grid_search.best_params_)
print("AdaBoosting best model feature importances:", adb_best_model.feature_importances_)


AdaBoosting Best Model: AdaBoostRegressor(learning_rate=0.1, n_estimators=100, random_state=42)
AdaBoosting MSE: 7.411957954582061
AdaBoosting R2: 0.8504552814983322
AdaBoosting cross validation score: [-7.60317368 -7.35121774 -7.38626191 -7.4621011  -7.37880034]


AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'

In [58]:
import joblib
import pickle as pkl

# Assuming `gb_grid_search` is your trained model
final_model = gb_grid_search

# Save the trained model using joblib
joblib.dump(final_model, '/content/drive/MyDrive/Colab Notebooks/gb_grid_search_joblib.pkl')



['/content/drive/MyDrive/Colab Notebooks/gb_grid_search_joblib.pkl']